In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing
from aif360.metrics import ClassificationMetric
from aif360.datasets import BinaryLabelDataset
import gc

# Load the Adult dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 
                'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']
df = pd.read_csv(url, names=column_names, skipinitialspace=True, na_values='?')

# Preprocess the data
df = df.dropna()
df['income'] = df['income'].map({'>50K': 1, '<=50K': 0})
df['sex'] = df['sex'].map({'Male': 1, 'Female': 0})

# Encode categorical variables
le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = le.fit_transform(df[column])

# Split the data
X = df.drop('income', axis=1)
y = df['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("Original Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

# Create a BinaryLabelDataset for fairness analysis
dataset = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1), label_names=['income'], protected_attribute_names=['sex'])

# Measure bias in the predictions
metric = ClassificationMetric(dataset, dataset, unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
print(f"Disparate Impact Before Mitigation: {metric.disparate_impact():.4f}")

# Post-processing with Calibrated Equalized Odds
postprocessor = CalibratedEqOddsPostprocessing(unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
dataset_pred = dataset.copy()
dataset_pred.scores = model.predict_proba(X_test)[:, 1].reshape(-1, 1)
dataset_postprocessed = postprocessor.fit(dataset, dataset_pred).predict(dataset_pred)

# Evaluate post-processed predictions
metric_post = ClassificationMetric(dataset, dataset_postprocessed, unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
print(f"Disparate Impact After Mitigation: {metric_post.disparate_impact():.4f}")


/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migratin

Original Model Performance:
Accuracy: 0.8528
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      4503
           1       0.74      0.64      0.69      1530

    accuracy                           0.85      6033
   macro avg       0.81      0.78      0.80      6033
weighted avg       0.85      0.85      0.85      6033

Disparate Impact Before Mitigation: 0.3524
Disparate Impact After Mitigation: 0.0000


In [13]:
!pip install seaborn

In [21]:
print("Shape of y_test_np:", y_test_np.shape)
print("Shape of y_pred_np:", y_pred_np.shape)
print("Shape of male_indices_np:", male_indices_np.shape)
print("Shape of female_indices_np:", female_indices_np.shape)
print("Shape of y_pred_post:", y_pred_post.shape)

print("\nType of y_test_np:", type(y_test_np))
print("Type of y_pred_np:", type(y_pred_np))
print("Type of male_indices_np:", type(male_indices_np))
print("Type of female_indices_np:", type(female_indices_np))
print("Type of y_pred_post:", type(y_pred_post))

Shape of y_test_np: (6033,)
Shape of y_pred_np: (6033,)
Shape of male_indices_np: (6033,)
Shape of female_indices_np: (6033,)
Shape of y_pred_post: (6033,)

Type of y_test_np: <class 'numpy.ndarray'>
Type of y_pred_np: <class 'numpy.ndarray'>
Type of male_indices_np: <class 'numpy.ndarray'>
Type of female_indices_np: <class 'numpy.ndarray'>
Type of y_pred_post: <class 'numpy.ndarray'>


In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(y_true, y_pred, title):
    # Ensure inputs are 1D numpy arrays
    y_true = np.ravel(y_true)
    y_pred = np.ravel(y_pred)
    
    cm = confusion_matrix(y_true, y_pred)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xlabel='Predicted label',
           ylabel='True label',
           title=title)

    # Loop over data dimensions and create text annotations
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    fig.tight_layout()
    plt.show()

# Now use this function for plotting
plot_confusion_matrix(y_test_np, y_pred_np, "Overall Confusion Matrix (Before Mitigation)")
plot_confusion_matrix(y_test_np[male_indices_np], y_pred_np[male_indices_np], "Confusion Matrix for Males (Before Mitigation)")
plot_confusion_matrix(y_test_np[female_indices_np], y_pred_np[female_indices_np], "Confusion Matrix for Females (Before Mitigation)")

# After mitigation
y_pred_post = dataset_postprocessed.labels.ravel()
plot_confusion_matrix(y_test_np[male_indices_np], y_pred_post[male_indices_np], "Confusion Matrix for Males (After Mitigation)")
plot_confusion_matrix(y_test_np[female_indices_np], y_pred_post[female_indices_np], "Confusion Matrix for Females (After Mitigation)")

def plot_probability_distribution(probs_male, probs_female, title):
    plt.figure(figsize=(10, 6))
    plt.hist(probs_male, bins=50, alpha=0.5, label='Males', density=True)
    plt.hist(probs_female, bins=50, alpha=0.5, label='Females', density=True)
    plt.title(title)
    plt.xlabel("Predicted Probability of Income > $50K")
    plt.ylabel("Density")
    plt.legend()
    plt.show()

# Before mitigation
probs_male = model.predict_proba(X_test[male_indices])[:, 1]
probs_female = model.predict_proba(X_test[female_indices])[:, 1]
plot_probability_distribution(probs_male, probs_female, "Probability Distribution Before Mitigation")

# After mitigation
probs_male_post = dataset_postprocessed.scores[male_indices_np].ravel()
probs_female_post = dataset_postprocessed.scores[female_indices_np].ravel()
plot_probability_distribution(probs_male_post, probs_female_post, "Probability Distribution After Mitigation")

from sklearn.metrics import classification_report

def print_metrics(y_true, y_pred, group_name):
    print(f"Metrics for {group_name}:")
    print(classification_report(y_true, y_pred))
    print()

# Performance metrics by gender
print_metrics(y_test_np[male_indices_np], y_pred_np[male_indices_np], "Males (Before Mitigation)")
print_metrics(y_test_np[female_indices_np], y_pred_np[female_indices_np], "Females (Before Mitigation)")
print_metrics(y_test_np[male_indices_np], y_pred_post[male_indices_np], "Males (After Mitigation)")
print_metrics(y_test_np[female_indices_np], y_pred_post[female_indices_np], "Females (After Mitigation)")

ValueError: object __array__ method not producing an array

<Figure size 800x600 with 2 Axes>

ValueError: object __array__ method not producing an array

<Figure size 800x600 with 2 Axes>

ValueError: object __array__ method not producing an array

...

<Figure size 1000x600 with 1 Axes>

ValueError: object __array__ method not producing an array

<Figure size 1000x600 with 1 Axes>

Metrics for Males (Before Mitigation):
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      2764
           1       0.75      0.66      0.70      1308

    accuracy                           0.82      4072
   macro avg       0.80      0.78      0.78      4072
weighted avg       0.81      0.82      0.81      4072


Metrics for Females (Before Mitigation):
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      1739
           1       0.72      0.56      0.63       222

    accuracy                           0.93      1961
   macro avg       0.83      0.77      0.79      1961
weighted avg       0.92      0.93      0.92      1961


Metrics for Males (After Mitigation):
              precision    recall  f1-score   support

           0       0.85      0.89      0.87      2764
           1       0.75      0.67      0.71      1308

    accuracy                           0.82      4072
   macro

/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [22]:
from sklearn.metrics import confusion_matrix
import numpy as np

def print_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    print(f"\n{title}")
    print(cm)

# Before mitigation
print_confusion_matrix(y_test_np, y_pred_np, "Overall Confusion Matrix (Before Mitigation)")
print_confusion_matrix(y_test_np[male_indices_np], y_pred_np[male_indices_np], "Confusion Matrix for Males (Before Mitigation)")
print_confusion_matrix(y_test_np[female_indices_np], y_pred_np[female_indices_np], "Confusion Matrix for Females (Before Mitigation)")

# After mitigation
print_confusion_matrix(y_test_np[male_indices_np], y_pred_post[male_indices_np], "Confusion Matrix for Males (After Mitigation)")
print_confusion_matrix(y_test_np[female_indices_np], y_pred_post[female_indices_np], "Confusion Matrix for Females (After Mitigation)")

def print_prob_distribution_stats(probs, group):
    print(f"\nProbability Distribution Statistics for {group}:")
    print(f"Mean: {np.mean(probs):.4f}")
    print(f"Median: {np.median(probs):.4f}")
    print(f"Std Dev: {np.std(probs):.4f}")
    print(f"Min: {np.min(probs):.4f}")
    print(f"Max: {np.max(probs):.4f}")

# Before mitigation
probs_male = model.predict_proba(X_test[male_indices])[:, 1]
probs_female = model.predict_proba(X_test[female_indices])[:, 1]
print_prob_distribution_stats(probs_male, "Males (Before Mitigation)")
print_prob_distribution_stats(probs_female, "Females (Before Mitigation)")

# After mitigation
probs_male_post = dataset_postprocessed.scores[male_indices_np].ravel()
probs_female_post = dataset_postprocessed.scores[female_indices_np].ravel()
print_prob_distribution_stats(probs_male_post, "Males (After Mitigation)")
print_prob_distribution_stats(probs_female_post, "Females (After Mitigation)")

from sklearn.metrics import classification_report

def print_metrics(y_true, y_pred, group_name):
    print(f"Metrics for {group_name}:")
    print(classification_report(y_true, y_pred))
    print()

# Performance metrics by gender
print_metrics(y_test_np[male_indices_np], y_pred_np[male_indices_np], "Males (Before Mitigation)")
print_metrics(y_test_np[female_indices_np], y_pred_np[female_indices_np], "Females (Before Mitigation)")
print_metrics(y_test_np[male_indices_np], y_pred_post[male_indices_np], "Males (After Mitigation)")
print_metrics(y_test_np[female_indices_np], y_pred_post[female_indices_np], "Females (After Mitigation)")


Overall Confusion Matrix (Before Mitigation)
[[4161  342]
 [ 546  984]]

Confusion Matrix for Males (Before Mitigation)
[[2471  293]
 [ 448  860]]

Confusion Matrix for Females (Before Mitigation)
[[1690   49]
 [  98  124]]

Confusion Matrix for Males (After Mitigation)
[[2467  297]
 [ 434  874]]

Confusion Matrix for Females (After Mitigation)
[[1739    0]
 [ 222    0]]

Probability Distribution Statistics for Males (Before Mitigation):
Mean: 0.3215
Median: 0.2100
Std Dev: 0.3246
Min: 0.0000
Max: 1.0000

Probability Distribution Statistics for Females (Before Mitigation):
Mean: 0.1232
Median: 0.0200
Std Dev: 0.2284
Min: 0.0000
Max: 0.9900

Probability Distribution Statistics for Males (After Mitigation):
Mean: 0.3215
Median: 0.2100
Std Dev: 0.3246
Min: 0.0000
Max: 1.0000

Probability Distribution Statistics for Females (After Mitigation):
Mean: 0.1132
Median: 0.1132
Std Dev: 0.0000
Min: 0.1132
Max: 0.1132
Metrics for Males (Before Mitigation):
              precision    recall  f1-sc

/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [15]:
print("Shape of y_test:", y_test.shape)
print("Shape of y_pred:", y_pred.shape)
print("Shape of male_indices:", male_indices.shape)
print("Shape of female_indices:", female_indices.shape)

print("\nType of y_test:", type(y_test))
print("Type of y_pred:", type(y_pred))
print("Type of male_indices:", type(male_indices))
print("Type of female_indices:", type(female_indices))

Shape of y_test: (6033,)
Shape of y_pred: (6033,)
Shape of male_indices: (6033,)
Shape of female_indices: (6033,)

Type of y_test: <class 'pandas.core.series.Series'>
Type of y_pred: <class 'numpy.ndarray'>
Type of male_indices: <class 'pandas.core.series.Series'>
Type of female_indices: <class 'pandas.core.series.Series'>


In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# Separate predictions by gender
male_indices = X_test['sex'] == 1
female_indices = X_test['sex'] == 0

# Original predictions by gender
y_pred_male = y_pred[male_indices]
y_pred_female = y_pred[female_indices]
y_test_male = y_test[male_indices]
y_test_female = y_test[female_indices]

# Post-processed predictions by gender
y_post_male = dataset_postprocessed.labels[male_indices]
y_post_female = dataset_postprocessed.labels[female_indices]

# Function to print performance metrics
def print_metrics(y_true, y_pred, group):
    print(f"\nPerformance for {group}:")
    print(f"Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred):.4f}")
    print(f"F1-Score: {f1_score(y_true, y_pred):.4f}")

# Original performance metrics
print_metrics(y_test_male, y_pred_male, "Males (Original)")
print_metrics(y_test_female, y_pred_female, "Females (Original)")

# Post-processed performance metrics
print_metrics(y_test_male, y_post_male, "Males (Post-Processed)")
print_metrics(y_test_female, y_post_female, "Females (Post-Processed)")

# Plot calibration curves for both groups
def plot_calibration_curve(y_true, y_prob, group):
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=10)
    plt.plot(prob_pred, prob_true, marker='o', label=group)

plt.figure(figsize=(10, 6))
plot_calibration_curve(y_test_male, model.predict_proba(X_test[male_indices])[:, 1], "Males (Original)")
plot_calibration_curve(y_test_female, model.predict_proba(X_test[female_indices])[:, 1], "Females (Original)")
plot_calibration_curve(y_test_male, dataset_postprocessed.scores[male_indices], "Males (Post-Processed)")
plot_calibration_curve(y_test_female, dataset_postprocessed.scores[female_indices], "Females (Post-Processed)")

plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
plt.title('Calibration Curves by Gender')
plt.legend()
plt.show()



Performance for Males (Original):
Precision: 0.7459
Recall: 0.6575
F1-Score: 0.6989

Performance for Females (Original):
Precision: 0.7168
Recall: 0.5586
F1-Score: 0.6278

Performance for Males (Post-Processed):
Precision: 0.7464
Recall: 0.6682
F1-Score: 0.7051

Performance for Females (Post-Processed):
Precision: 0.0000
Recall: 0.0000
F1-Score: 0.0000


/Users/ttanaka/miniconda3/envs/new_env_name/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ValueError: object __array__ method not producing an array

<Figure size 1000x600 with 1 Axes>